In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import xarray as xr
import os
from pathlib import Path
from numpy import nanmedian
import scipy
import scipy.signal
import math
from numpy.fft import fft2
import cmocean
import matplotlib
plt.rcParams["figure.figsize"] = 10, 8
%matplotlib inline

In [2]:
current_path = os.getcwd()
current_path

'/home/luisfigueiredo/JUNO/notebooks'

In [3]:

data_folder = os.path.join(current_path,"../data")
data_folder

'/home/luisfigueiredo/JUNO/notebooks/../data'

In [7]:
nc_path = os.path.join(data_folder, "AQUA_MODIS.20220209T133501.L2.SST.NRT.nc")

In [8]:
nc_path

'/home/luisfigueiredo/JUNO/notebooks/../data/AQUA_MODIS.20220209T133501.L2.SST.NRT.nc'

In [10]:

ds = xr.open_dataset(nc_path, group='geophysical_data')
df = ds.to_dataframe()

In [11]:
df

sst  qual_sst  flags_sst  bias_sst  stdv_sst  \
number_of_lines pixels_per_line                                                 
0               0                NaN       4.0        1.0     -18.0     -18.0   
                1                NaN       4.0        1.0     -18.0     -18.0   
                2                NaN       4.0        1.0     -18.0     -18.0   
                3                NaN       4.0        1.0     -18.0     -18.0   
                4                NaN       4.0        1.0     -18.0     -18.0   
...                              ...       ...        ...       ...       ...   
2029            1349             NaN       4.0        1.0     -18.0     -18.0   
                1350             NaN       4.0        1.0     -18.0     -18.0   
                1351             NaN       4.0        1.0     -18.0     -18.0   
                1352             NaN       4.0        1.0     -18.0     -18.0   
                1353             NaN       4.0        1.0     -18.0     -18.0   

                                 sstref  l2_flags  
number_of_lines pixels_per_line                    
0               0                   NaN        50  
                1                   NaN        50  
                2                   NaN        50  
                3                   NaN        50  
                4                   NaN        50  
...                                 ...       ...  
2029            1349               -3.0  33619984  
                1350               -3.0  33619984  
                1351               -3.0  33619984  
                1352               -3.0  33619984  
                1353               -3.0  33619984  

[2748620 rows x 7 columns]

### Código: Load das variaveis MATLAB

In [ ]:

## LOAD das variáveis necessários do netCDF (Este exemplo é com imagens
# MODIS - os caminhos podem ser diferentes)
lon=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/navigation_data/longitude');
lat=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/navigation_data/latitude');
sst=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/geophysical_data/sst');
qual_sst=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/geophysical_data/qual_sst');

# factor de qualidade para as imagens MODIS - esta linha é para comentar no
# caso do MUR e/ou modelo
sst(qual_sst>1)=NaN;

# Figura com os dados SST - grelha irregular L2
pcolor(lon,lat,sst); shading flat; colorbar

# interpolar dados para uma grelha antes - também não é necessário nos dados L3, L4 ou Modelo

Z1=double(sst);
X1=double(lon);
Y1=double(lat);
X1(isnan(X1))=0;  #caso lon tenha NaN values, estes são convertidos para zeros
Y1(isnan(Y1))=0;  #caso lat tenha NaN values, estes são convertidos para zeros

[X,Y]=meshgrid(-27.296:.02:7.4435,32.2344:0.02:53.7133);

Z=griddata(X1,Y1,Z1,X,Y);

#### Código: LOAD das variaveis em Python (passo a passo)

In [165]:
lon = np.array([1,2,3,4,5,6])
lat = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
sst = np.arange(1,91, dtype=np.double)#.reshape(6,15) 

lon[np.isnan(lon)] = 0    #caso lon tenha NaN values, estes são convertidos para zeros
lat[np.isnan(lat)] = 0    #caso lat tenha NaN values, estes são convertidos para zeros

# corresponde ao [X,Y]=meshgrid(-27.296:.02:7.4435,32.2344:0.02:53.7133); do MATLAB
# No entanto como estamos a usar valores dummy isto para já não serve para nada
#X = np.arange(-27.296, 7.4435, 0.02)
#Y = np.arange(32.2344, 53.7133, 0.02) 
#X, Y = np.meshgrid(X, Y)

#Código de cima mas com dummy variables
X1 = np.linspace(-27.296, 7.4435, 6)
Y1 = np.linspace(32.2344, 53.7133, 15)
X1, Y1 = np.meshgrid(X1, Y1)

from scipy.interpolate import griddata
#Z = griddata([lat, lon], sst, (X1, Y1), method='linear') #NÃO CONSIGO POR ISTO A FUNCIONAR


### Código: Definição da janela MATLAB

In [ ]:
#####################################

# Definição da janela

head(1:2) = [-27.296 7.4435];             #cria variavel head com esses 2 valores nas posiçoes 1 e 2
head(3:4) = [32.2344 53.7133];
z_dim=size(Z); #dimensões da matriz Z
z_actual_range = double([min(Z(:)) max(Z(:))]); #range dos dados
	nx = z_dim(end);		ny = z_dim(end-1);
    node_offset = 0;	
 
	head(5:7) = double([z_actual_range node_offset]);
	head(8) = diff(head(1:2)) / (nx - ~node_offset);
	head(9) = diff(head(3:4)) / (ny - ~node_offset);
    
############################################ 

#### Código: Definição da janela em Python (passo a passo)

In [273]:
head = np.array([-27.296, 7.4435])
head = np.append(head, [32.2344, 53.7133])
#create dummy array Z to test 
Z = np.array([[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],
             [1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,10],[1,2,3,4,5,6],[1,1,1,1,1,1]])

z_dim = Z.shape    #dimensões da matriz Z

z_actual_range = np.array([np.min(Z[:]), np.max(Z[:])])    #range dos dados
nx = z_dim[1]
ny = z_dim[0]
node_offset = 0

#Solução para o (~node_offset) em MATLAB
if node_offset == 0:
    logical = 1
else:
    logical = 0

head = np.append(head, np.array([z_actual_range[0], z_actual_range[1] , node_offset]))
head = np.append(head, np.array((head[1]- head[0])/(nx - logical)))     
head = np.append(head, np.array((head[3]- head[2])/(ny - logical)))


In [274]:
head

array([-27.296     ,   7.4435    ,  32.2344    ,  53.7133    ,
         1.        ,  10.        ,   0.        ,   6.9479    ,
         2.38654444])

### Código: Definição da parametros MATLAB

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  

% Window size 32 × 32

% Window clarity (% of data have to present in the window) 75%
% Overlap 50%

% The ratio of variance between the two populations to the
% variance within the populations θ(τopt )% 0.76

% Gradient ratio 0.9

% Cohesion coefficients [C1, C2, C] [0.9, 0.9, 0.92]
% Mean temperature difference between the two populations (surface waters)
% .25

   %cayula
    minPopProp = 0.20;				% 
	minPopMeanDiff = 0.4;
	minTheta = 0.70;
	minSinglePopCohesion = 0.90;
	minGlobalPopCohesion = 0.70;
   %nieto
%     minPopProp = 0.15;				% 
% 	minPopMeanDiff = 0.4;
% 	minTheta = 0.70;
% 	minSinglePopCohesion = 0.90;
% 	minGlobalPopCohesion = 0.70;
%%%%%%%%%%%%%%%%%%%%%%%

#### Código: Definição da parametros Python (passo a passo)

In [275]:
################################# 

# Window size 32 × 32

# Window clarity (% of data have to present in the window) 75%
# Overlap 50%

# The ratio of variance between the two populations to the
# variance within the populations θ(τopt )% 0.76

# Gradient ratio 0.9

# Cohesion coefficients [C1, C2, C] [0.9, 0.9, 0.92]
# Mean temperature difference between the two populations (surface waters)
# .25

#cayula
minPopProp = 0.20
minPopMeanDiff = 0.4
minTheta = 0.70
minSinglePopCohesion = 0.90
minGlobalPopCohesion = 0.70

#nieto
# minPopProp = 0.15
# minPopMeanDiff = 0.4
# minTheta = 0.70
# minSinglePopCohesion = 0.90
# minGlobalPopCohesion = 0.70

##################################